Download genes from: https://www.oncokb.org/cancer-genes (read the .tsv)  
Then filter via the Vogelstein et al. (2013). column   
Then create a parquet with similar columns to kaggle/input/oncokb-cancer-gene-list/mutations_3.parquet

From BiolSys - zajęcia:

## 1. Przygotowanie danych:

## OncoKB database:

In [1]:
import pandas as pd

oncokb_df = pd.read_csv('/kaggle/input/oncokb-cancer-gene-list/cancerGeneList.tsv', sep="\t")
oncokb_df
genes_list = oncokb_df['Hugo Symbol']

## TCGA database:

In [2]:
# tutaj można zamiast tego zrobić list.join() albo coś takiego
list(genes_list)
genes_list_str = "("
for gene in genes_list:
    genes_list_str += f"'{gene}', "
genes_list_str = genes_list_str.strip(", ")
genes_list_str += ")"


In [3]:
from google.cloud import bigquery
import pandas as pd
# Tworzenie klienta BigQuery
client = bigquery.Client()

# Zapytanie SQL
query = """SELECT *
FROM `isb-cgc-bq.TCGA.masked_somatic_mutation_hg38_gdc_current`
WHERE Variant_Type IN ('SNP', 'DEL', 'INS') AND 
Hugo_Symbol IN 
"""
query = query + genes_list_str + ";"
query_job = client.query(query)
df = query_job.to_dataframe()


Using Kaggle's public dataset BigQuery integration.


/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [4]:
df

,project_short_name,case_barcode,primary_site,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,...,callers,file_gdc_id,muse,mutect2,pindel,varscan2,sample_barcode_tumor,sample_barcode_normal,aliquot_barcode_tumor,aliquot_barcode_normal
0,TCGA-OV,TCGA-04-1347,Ovary,AGO2,27161,BI,GRCh38,chr8,140585293,140585293,...,muse;mutect2;varscan2,3b23fc2d-23e1-4847-ac11-edc83bce1efa,Yes,Yes,No,Yes,TCGA-04-1347-01A,TCGA-04-1347-11A,TCGA-04-1347-01A-01W-0486-08,TCGA-04-1347-11A-01W-0487-08
1,TCGA-GBM,TCGA-06-5416,Brain,NCOA2,10499,BI,GRCh38,chr8,70124775,70124775,...,muse;varscan2,99c228a0-02df-46f2-a095-68e72e53b159,Yes,No,No,Yes,TCGA-06-5416-01A,TCGA-06-5416-10A,TCGA-06-5416-01A-01D-1486-08,TCGA-06-5416-10A-01D-1486-08
2,TCGA-GBM,TCGA-06-5858,Brain,PRDM16,63976,BI,GRCh38,chr1,3412264,3412264,...,muse;mutect2;varscan2,a7280605-858e-4595-82db-e7508d9a8f57,Yes,Yes,No,Yes,TCGA-06-5858-01A,TCGA-06-5858-10A,TCGA-06-5858-01A-01D-1696-08,TCGA-06-5858-10A-01D-1696-08
3,TCGA-GBM,TCGA-16-0848,Brain,ADGRG4,139378,BI,GRCh38,chrX,136323214,136323214,...,muse;mutect2,192041a9-6ff6-4e10-94c8-12a31849a790,Yes,Yes,No,No,TCGA-16-0848-01A,TCGA-16-0848-10A,TCGA-16-0848-01A-01W-0424-08,TCGA-16-0848-10A-01D-1492-08
4,TCGA-LUAD,TCGA-17-Z017,Bronchus and lung,CBL,867,BI,GRCh38,chr11,119274922,119274922,...,mutect2;varscan2,0e9ceaa9-c20f-45a1-a55e-7ef515c0b15d,No,Yes,No,Yes,TCGA-17-Z017-01A,TCGA-17-Z017-11A,TCGA-17-Z017-01A-01W-0746-08,TCGA-17-Z017-11A-01W-0746-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244270,TCGA-READ,TCGA-F5-6814,Rectum,DDX5,1655,BCM,GRCh38,chr17,64500565,64500565,...,muse;varscan2,18118987-2447-4e6a-9516-69be8f87d03f,Yes,No,No,Yes,TCGA-F5-6814-01A,TCGA-F5-6814-10A,TCGA-F5-6814-01A-31D-1924-10,TCGA-F5-6814-10A-01D-1924-10
244271,TCGA-READ,TCGA-F5-6814,Rectum,KEL,3792,BCM,GRCh38,chr7,142946252,142946252,...,muse;mutect2;varscan2,18118987-2447-4e6a-9516-69be8f87d03f,Yes,Yes,No,Yes,TCGA-F5-6814-01A,TCGA-F5-6814-10A,TCGA-F5-6814-01A-31D-1924-10,TCGA-F5-6814-10A-01D-1924-10
244272,TCGA-STAD,TCGA-HU-A4GQ,Stomach,SDHA,6389,BI,GRCh38,chr5,240386,240386,...,muse;mutect2;varscan2,130a54ce-e31d-456e-8681-5edb347f9761,Yes,Yes,No,Yes,TCGA-HU-A4GQ-01A,TCGA-HU-A4GQ-10A,TCGA-HU-A4GQ-01A-11D-A25D-08,TCGA-HU-A4GQ-10A-01D-A25E-08
244273,TCGA-ACC,TCGA-OR-A5LJ,Adrenal gland,ARHGAP35,2909,BCM,GRCh38,chr19,46920741,46920741,...,muse;mutect2;varscan2,10a0ec35-8ffc-4e59-9d88-f39d5e388866,Yes,Yes,No,Yes,TCGA-OR-A5LJ-01A,TCGA-OR-A5LJ-10A,TCGA-OR-A5LJ-01A-11D-A29I-10,TCGA-OR-A5LJ-10A-01D-A29L-10


## Odczyt z formatu parquet i integracja danych w pandas dataframe

In [5]:
import pandas as pd
import re

df = pd.read_parquet('/kaggle/input/oncokb-cancer-gene-list/mutations_3.parquet')
df.head()
df = df[(df['Variant_Type'] == 'SNP') | (df['Variant_Type'] == 'DEL') | (df['Variant_Type'] == 'INS')]
df['Hugo_Symbol'] = df['Hugo_Symbol'].fillna(value='')


3. Format Parquet  
• Zapisz oczyszczone dane mutacji w formacie Parquet.  
• Pobierz profil ekspresji RNA (TCGA RNA-seq), usuń duplikaty i zapisz w
Parquet.  
5. Charakterystyka etykiet  
• Zbiór etykiet primary site jest niezbalansowany – różne klasy maja różna
liczebność. Zbadaj jaką liczebność mają różne klasy, np. tabelka liczności primary site.

## 2. Inżynieria cech

In [6]:
!pip install biopython pyranges

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 79.3 MB/s eta 0:00:00


In [7]:
import re 
df = df.rename(columns={'Start_position': 'Start'})
df = df.rename(columns={'End_position': 'End'})

In [8]:
!wget http://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
!gunzip hg38.fa.gz

--2025-06-26 10:19:20--  http://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 983659424 (938M) [application/x-gzip]
Saving to: ‘hg38.fa.gz’

hg38.fa.gz          100%[===================>] 938.09M  62.9MB/s    in 16s     

2025-06-26 10:19:36 (59.9 MB/s) - ‘hg38.fa.gz’ saved [983659424/983659424]



In [9]:
import pyranges as pr
import pandas as pd
from Bio import SeqIO

gr = pr.PyRanges(df)
fasta_sequences = SeqIO.to_dict(SeqIO.parse("/kaggle/working/hg38.fa", "fasta"))

## Ekstrakcja motywów mutacyjnych:

In [10]:
def get_cosmic_notation(row, fasta_dict):
    chrom = row.Chromosome  # upewniamy się, że chromosom nie ma prefiksu "chr"
    start = int(row.Start)
    end = int(row.End)
    alt = str(row.Tumor_Seq_Allele2) if pd.notnull(row.Tumor_Seq_Allele2) else None
    variant_type = row.Variant_Type
    cosmic_notation = None

    try:
        seq = fasta_dict[chrom].seq
        context_seq = seq[start-2:end+1].upper()  # np. G[C>T]A -> 1 baza przed i po
        ref_base = seq[start-1:end].upper()  

        # COSMIC-style notacja
        if variant_type == 'SNP':
            # np. G[C>T]A
            cosmic_notation = f"{context_seq[0]}[{ref_base}>{alt}]{context_seq[-1]}"

        elif variant_type == 'DEL':
            if end > start:
                deleted_seq = seq[start-1:end].upper()
            else:
                deleted_seq = ref_base
            cosmic_notation = f"{context_seq[0]}[del{len(deleted_seq)}]{context_seq[-1]}"

        elif variant_type == 'INS':
            inserted_seq = alt
            cosmic_notation = f"{context_seq[0]}[ins{len(inserted_seq)}]{context_seq[-1]}"

        else:
            cosmic_notation = f"UnknownVariantType:{variant_type}"

    except Exception as e:
        cosmic_notation = f"ERROR: {e}"

    return cosmic_notation


In [11]:
df['Mutational_Motif'] = df.apply(lambda row: get_cosmic_notation(row, fasta_sequences), axis=1)

## Genomiczne binowanie:

In [12]:
df['Genomic_Bin'] = df['Chromosome'].astype(str) + '_' + (
    ((df['Start'] - 1) // 1_000_000) * 1_000_000
).astype(str)

## Normalizacja mutacji do pitymidyn, komplementacja:

In [13]:
complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}

def normalize_mutation(mutation):
    match = re.match(r"([ATGC])\[([ATGC]>[ATGC])\]([ATGC])", mutation)
    if match:
        ref, change, alt = match.groups()
        ref_from, ref_to = change.split('>')

        if ref_from not in ['C', 'T']:
            ref_from = complement[ref_from]
            ref_to = complement[ref_to]
            ref = complement[alt]
            alt = complement[ref]

        return f"{ref}[{ref_from}>{ref_to}]{alt}"

    return mutation

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer

mutations = ['A[A>C]A', 'G[G>A]T', 'C[C>T]G', 'T[T>G]T']

df['MutationType'] = df['Mutational_Motif'].apply(normalize_mutation)

top_sites = df['primary_site'].value_counts().nlargest(24).index

filtered_df = df[df['primary_site'].isin(top_sites)]

filtered_df['primary_site_cat'] = filtered_df['primary_site'].astype('category').cat.codes


grouped_mutations_df = filtered_df.groupby('case_barcode').agg({
    'Mutational_Motif': list,
    'Genomic_Bin': list,
    'Hugo_Symbol': list,
    'primary_site': 'first',
    'primary_site_cat': 'first',
}).reset_index()



2025-06-26 10:21:18.955590: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750933279.225554      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750933279.303104      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_13/1731204343.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['primary_site_cat'] = filtered_df['primary_site'].astype('category').cat.codes


### Usunięcie duplikatów ekspresji:

In [15]:
df_expressions = pd.read_parquet('/kaggle/input/oncokb-cancer-gene-list/expression_2.parquet')

df_expressions = df_expressions.drop_duplicates(subset=['gene_name', 'case_barcode'])

In [16]:
grouped_expression_df = df_expressions.groupby('case_barcode').agg({
    'gene_name': list,
    'tpm_unstranded': list,
}).reset_index()


grouped_df = pd.merge(grouped_mutations_df, grouped_expression_df, on="case_barcode", suffixes=('_mutation', '_expression'))


tokenizer_bin = Tokenizer(filters='', lower=False, split=' ', oov_token="<OOV>")
tokenizer_bin.fit_on_texts(df['Genomic_Bin'].astype(str).to_list())

tokenizer_gene = Tokenizer(filters='', lower=False, split=' ', oov_token="<OOV>")
tokenizer_gene.fit_on_texts(df['Hugo_Symbol'].astype(str).to_list())


### Tokenizacja genów, motywów i binów:

In [17]:
import numpy as np
import tensorflow as tf

def to_matrix(tokenizer, texts):
    sequences = tokenizer.texts_to_sequences(texts)
    count_vectors = np.zeros((len(texts), len(tokenizer.word_index) + 1))

    for i, seq in enumerate(sequences):
        for token in seq:
            count_vectors[i, token] += 1

    return count_vectors

In [18]:
def prepare_data_for(df, tokenizer_bin, tokenizer_gene):
    X_bin = to_matrix(tokenizer_bin, df['Genomic_Bin'])
    X_gene = to_matrix(tokenizer_gene, df['Hugo_Symbol'])
    X_rna = np.array(df['tpm_unstranded'].to_list())

    return X_bin, X_gene, X_rna

2. Ekspresja RNA  
• Normaliacja poprzez logarytmowanie (log1p).

In [19]:
X_genomic_bin, X_gene, X_rna = prepare_data_for(grouped_df, tokenizer_bin, tokenizer_gene)


### Konkatenacja wektorów:  
Potrzebny jeszcze wektor motywów

In [20]:
X = np.concatenate([X_genomic_bin, X_gene, X_rna], axis=1)


In [21]:
X.shape

(13162, 3243)

• Przygotuj etykiety: y primary site.

## 3. Budowa i trening modelu  
### 1. Architektura  
• Wejścia dla każdej grupy cech (warstwy Dense).  
• Laczenie (Concatenate) oraz warstwy ukryte.  
• Wyjście: softmax nad klasami lokalizacji pierwotnej.  
### 2. Trening  
• Podzial na zbiór treningowy/testowy 80/20.  
• Optymalizator: Adam, strata: categorical crossentropy.  
• Monitorowanie loss i accuracy.  
• Uwzglednienie niezbalansowanych etykiet.  
## 4. Ocena overfittingu  
### 1. Wizualizacja uczenia  
• Krzywe loss/accuracy dla zbioru treningowego i walidacyjnego.  
### 2. Metryki  
• Accuracy, precision, recall, F1 na zbiorze walidacyjnym.  
### 3. Identyfikacja overfittingu  
• Nierozbieżność miedzy strata/train a strata/val.  
• Spadek lub stagnacja accuracy/val przy rosnacym accuracy/train.